In [1]:
pip install --upgrade sodapy

Requirement already up-to-date: sodapy in c:\users\nuran\anaconda3\lib\site-packages (2.2.0)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
from sodapy import Socrata
from google.cloud import bigquery
from google.oauth2 import service_account

In [2]:
key_path = r'C:\Users\nuran\Desktop\CIS9440\project\cis9440-375301-db3c117da2fe.json'

In [3]:
credentials = service_account.Credentials.from_service_account_file(key_path,
                                                                    scopes=["https://www.googleapis.com/auth/cloud-platform"],)
bigquery_client = bigquery.Client(credentials = credentials,
                                 project = credentials.project_id)

print(f"bigquery client name is: {bigquery_client}")
print(f"bigquery client data type is: {type(bigquery_client)}")

bigquery client name is: <google.cloud.bigquery.client.Client object at 0x0000024569819F40>
bigquery client data type is: <class 'google.cloud.bigquery.client.Client'>


In [4]:
dataset_id = 'cis9440-375301.Group12_Final_Project'   # PASTE THIS DATASET ID FROM ABOVE STEPS

dataset_id = dataset_id.replace(':', '.')
print(f"your dataset_id is: {dataset_id}")

your dataset_id is: cis9440-375301.Group12_Final_Project


In [9]:
df = pd.read_csv("Group12_cleaned_df.csv")

In [10]:
location_dim = df.copy()

In [11]:
location_dim.columns

Index(['School Name', 'Grade', 'Year', 'Mean Scale Score', '# Level 1',
       '% Level 1', '# Level 2', '% Level 2', '# Level 3', '% Level 3',
       '# Level 4', '% Level 4', '# Level 3+4', '% Level 3+4',
       'District Number', 'School Number', 'location_ID', 'level_ID',
       'year_ID', 'Borough'],
      dtype='object')

In [12]:
location_dim = location_dim[['location_ID',
                             'Borough',
                             'School Number',
                             'District Number']]

In [13]:
unique_row = ['location_ID']
location_dim = location_dim.drop_duplicates(subset = unique_row, keep = 'first')
location_dim = location_dim.reset_index(drop = True)
location_dim

,location_ID,Borough,School Number,District Number
0,100,Manhattan,15,1
1,101,Manhattan,15,1
2,102,Manhattan,15,1
3,103,Manhattan,15,1
4,104,Manhattan,15,1
...,...,...,...,...
57379,57479,Brooklyn,562,32
57380,57480,Brooklyn,562,32
57381,57481,Brooklyn,562,32
57382,57482,Brooklyn,562,32


In [14]:
levels_dim = df.copy()

In [15]:
levels_dim.columns

Index(['School Name', 'Grade', 'Year', 'Mean Scale Score', '# Level 1',
       '% Level 1', '# Level 2', '% Level 2', '# Level 3', '% Level 3',
       '# Level 4', '% Level 4', '# Level 3+4', '% Level 3+4',
       'District Number', 'School Number', 'location_ID', 'level_ID',
       'year_ID', 'Borough'],
      dtype='object')

In [16]:
levels_dim = levels_dim[['level_ID',
                             '% Level 1',
                             '% Level 2',
                             '% Level 3', 
                        '% Level 4', 
                        '% Level 3+4']]

In [17]:
unique_row = ['level_ID']
levels_dim = levels_dim.drop_duplicates(subset = unique_row, keep = 'first')
levels_dim = levels_dim.reset_index(drop = True)
levels_dim

,level_ID,% Level 1,% Level 2,% Level 3,% Level 4,% Level 3+4
0,200,51.85185242,40.74074173,7.407407284,0,7.407407284
1,201,55.55555725,44.44444275,0,0,0
2,202,56.25,31.25,12.5,0,12.5
3,203,50,30,20,0,20
4,204,37.0370369,29.62962914,25.92592621,7.407407284,33.33333206
...,...,...,...,...,...,...
57379,57579,62,28,8,2,10
57380,57580,75.78125,24.21875,0,0,0
57381,57581,60.43955994,36.26373672,3.296703339,0,3.296703339
57382,57582,67.04545593,28.40909004,3.409090996,1.136363626,4.545454502


In [21]:
year_dim = df

In [22]:
year_dim = year_dim[['year_ID','Year']]

In [23]:
unique_year = ["Year"]
year_dim = year_dim.drop_duplicates(subset = unique_year, keep = 'first')
year_dim = year_dim.reset_index(drop = True)
year_dim

,year_ID,Year
0,300,2013
1,301,2014
2,302,2015
3,303,2016
4,304,2017
5,305,2018
6,306,2019
7,307,2022


In [24]:
df.columns

Index(['School Name', 'Grade', 'Year', 'Mean Scale Score', '# Level 1',
       '% Level 1', '# Level 2', '% Level 2', '# Level 3', '% Level 3',
       '# Level 4', '% Level 4', '# Level 3+4', '% Level 3+4',
       'District Number', 'School Number', 'location_ID', 'level_ID',
       'year_ID', 'Borough'],
      dtype='object')

In [27]:
fact_table = df[['location_ID',
                   'level_ID',
                   'year_ID',
                'Mean Scale Score']]

fact_table

,location_ID,level_ID,year_ID,Mean Scale Score
0,100,200,300,289.2962952
1,101,201,301,285.1111145
2,102,202,302,281.8125
3,103,203,303,292.5
4,104,204,304,302.3703613
...,...,...,...,...
57379,57479,57579,57679,584.4299927
57380,57480,57580,57680,267.9453125
57381,57481,57581,57681,273.9890137
57382,57482,57582,57682,267.7954407


In [ ]:
def load_table_to_bigquery(df,
                          table_name,
                          dataset_id):

    dataset_id = dataset_id #change 301800 to match your project id

    dataset_ref = bigquery_client.dataset(dataset_id)
    job_config = bigquery.LoadJobConfig()
    job_config.autodetect = True
    job_config.write_disposition = "WRITE_TRUNCATE"

    upload_table_name = f"{dataset_id}.{table_name}"
    
    load_job = bigquery_client.load_table_from_dataframe(df,
                                                upload_table_name,
                                                job_config = job_config)
        
    print(f"""completed loading {table_name} -- {load_job}""")

In [ ]:
load_table_to_bigquery(df = location_dim,
                       table_name = "location_dim",
                       dataset_id = dataset_id)

In [ ]:
load_table_to_bigquery(df = location_dim,
                       table_name = "year_dim",
                       dataset_id = dataset_id)

In [ ]:
load_table_to_bigquery(df = location_dim,
                       table_name = "levels_dim",
                       dataset_id = dataset_id)